In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, when, rand, current_date, date_add, date_format
import os

spark = SparkSession.builder.appName("SyntheticData") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

# Функция для ввода положительного целого числа
def input_number(number):
    while True:
        try:
            user_input  = int(input(number))
            if user_input > 0:
                return user_input
            else:
                print("Пожалуйста, введите положительное целое число.")
        except ValueError:
            print("Пожалуйста, введите целое число.")

# Запрос числа строк для генерации от пользователя
number_row = input_number("Сколько строк вы хотите сгенерировать: ")

# Генерация данных для колонок id, name, city, email, age, salary, registration_date
data = [(i, f"Names_{i}", f"Cityname_{i}") for i in range(1, number_row + 1)]
df = spark.createDataFrame(data, ["id", "name", "city"])

df = df.withColumn("email", concat(col("name"), lit("@example."), when(col("id") % 2 == 0, "ru").otherwise("com"))) \
    .withColumn("age", (rand() * 78 + 18).cast("int")) \
    .withColumn("salary", (rand() * 150000 + 19242).cast("int")) \
    .withColumn("registration_date", date_add(current_date(), - (rand() * (col("age") - 18)).cast("int") * 365))

df.cache()  # Кеширование DataFrame для более быстрой обработки

# Замена 5% данных на значение NULL
columns = ["name", "email", "city", "age", "salary", "registration_date"]
df = df.select("id", *[when(rand() <= 0.05, None).otherwise(col(column)).alias(column) for column in columns])

# Установка текущей даты для имени файла
df_date = spark.sql("SELECT current_date()")
current_date = df_date.select(date_format("current_date", "yyyy-MM-dd")).first()[0]

# Путь к сохранению CSV файла
path = f"/home/jovyan/work/PySpark_test/config/temp/{current_date}-dev.csv"
path_rename = f"/home/jovyan/work/PySpark_test/config/more_csv/{current_date}-dev.csv"
# Запись DataFrame в CSV
df.coalesce(1).write.csv(path, header=True, mode="overwrite")  

# Переименование сгенерированного файла
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    if file.startswith("part-"):
        os.rename(file_path, path_rename)

# Удаление временных файлов и папки
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    os.remove(file_path)
    
os.rmdir(path)

df.show()

spark.stop()

Сколько строк вы хотите сгенерировать:  1000000


+---+--------+--------------------+-----------+---+------+-----------------+
| id|    name|               email|       city|age|salary|registration_date|
+---+--------+--------------------+-----------+---+------+-----------------+
|  1| Names_1| Names_1@example.com| Cityname_1| 86| 88966|       1992-08-08|
|  2| Names_2|  Names_2@example.ru| Cityname_2| 72| 66714|       1976-08-12|
|  3| Names_3| Names_3@example.com| Cityname_3| 47| 88701|       2016-08-02|
|  4| Names_4|  Names_4@example.ru| Cityname_4| 58| 85602|       2016-08-02|
|  5|    NULL| Names_5@example.com| Cityname_5| 94|  NULL|       1971-08-14|
|  6| Names_6|  Names_6@example.ru| Cityname_6| 77| 44771|       1982-08-11|
|  7| Names_7| Names_7@example.com| Cityname_7| 39| 65415|       2016-08-02|
|  8| Names_8|  Names_8@example.ru| Cityname_8| 18| 43731|       2024-07-31|
|  9| Names_9| Names_9@example.com| Cityname_9| 29|122625|       2016-08-02|
| 10|Names_10| Names_10@example.ru|Cityname_10| 49| 58761|       2004-08-05|